In [ ]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-06-25 18:05:10--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-06-25 18:05:10--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf3f24d42e20a52dc0e92f1e968.dl.dropboxusercontent.com/cd/0/inline/Bn5vr6rMCT1XlP-Ma_AiE5ZZQSEQvYmMZ80MPP5KbQo0Fu-mW-rJ8dcgp8oOkgewOGkRkT8t0GxxxPlKf5TJ0sK-AutxCkKaRD-d1id2Cxq_d5nf6fEcqNUDrslt6LH-ldK1Hqb8hl2dQYcazLtcSnjwef5qEsjYGerIiQ_-JuN7sQ/file# [following]
--2022-06-25 18:05:10--  https://ucf3f24d42e20a52dc0e92f1e968.dl.dropboxusercontent.com/cd/0/inline/Bn5vr6rMCT1XlP-Ma_AiE5ZZQSEQv

In [ ]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [ ]:
# Sentiment Analysis

In [ ]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [ ]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />', ' '), label)
  )

In [ ]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'first, someone mentioned here that because this was released in "limited" quantity it means that it SHOULD be bad...that is exactly what the "big five" Hollywood studios would like everyone to think so they can "pass" or "ignore" features that are not desirable, without loosing face or imagine by censoring them directly. to the point, this production has been released "limited" because is considered "unpatriotic" by certain individuals.  now i absolutely loved this feature; i find it way better then "Charlie Wilson\'s War" even if it is a "fictional" account of something that "never" happens but is always so OBVIOUS.this goes to anyone and everyone interested or affected by present American foreign policies, "home" or "aboard". the "turakistan" country and "the emerald city" are definitely trying to resemble Iraq and Baghdad just as much as the corporation "Tamerlane" goes for "haliburton" (with the v

In [ ]:
model = Sequential()

In [ ]:
model.add(Input(shape=(1,), dtype="string"))

In [ ]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_len
)

In [ ]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [ ]:
model.add(Embedding(max_tokens + 1, 128))

model.add(LSTM(64))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 61s 75ms/step - loss: 0.5573 - accuracy: 0.7059
Epoch 2/10
782/782 [==============================] - 58s 74ms/step - loss: 0.4420 - accuracy: 0.7979
Epoch 3/10
782/782 [==============================] - 57s 72ms/step - loss: 0.4111 - accuracy: 0.8143
Epoch 4/10
782/782 [==============================] - 56s 72ms/step - loss: 0.3915 - accuracy: 0.8236
Epoch 5/10
782/782 [==============================] - 57s 72ms/step - loss: 0.3677 - accuracy: 0.8373
Epoch 6/10
782/782 [==============================] - 57s 73ms/step - loss: 0.3522 - accuracy: 0.8443
Epoch 7/10
782/782 [==============================] - 57s 73ms/step - loss: 0.3345 - accuracy: 0.8532
Epoch 8/10
782/782 [==============================] - 57s 73ms/step - loss: 0.3300 - accuracy: 0.8576
Epoch 9/10
782/782 [==============================] - 57s 73ms/step - loss: 0.3003 - accuracy: 0.8700
Epoch 10/10
782/782 [==============================] - 57s 73ms/step - loss: 0.293

In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 19s 24ms/step - loss: 0.6062 - accuracy: 0.7830


[0.6062085032463074, 0.7829599976539612]

In [ ]:
text = "I loved the movie !"

In [ ]:
model.predict([text])

array([[0.86068225]], dtype=float32)